# Análise de Inteligência Comercial - FMCG Doces e Guloseimas
## ETL → EDA → Forecast → Dashboard Export

Neste notebook, realizamos:
1. ETL nas bases **Sell In** e **Sell Out**  
2. Análise Exploratória de Dados (EDA) básica e avançada  
3. Forecast de vendas (próxima seção)  
4. Exportação para dashboard  
---

In [12]:
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
from IPython.display import display, Markdown

# Definição de diretórios
BASE_DIR      = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
DATA_DIR      = BASE_DIR / 'data'
RAW_DIR       = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Mapeamento mês → número
mes_map = {
    'jan':1,'fev':2,'mar':3,'abr':4,'mai':5,'jun':6,
    'jul':7,'ago':8,'set':9,'out':10,'nov':11,'dez':12
}

def parse_mes_ano(x):
    s = str(x).strip().lower()
    if '/' in s:
        try:
            return datetime.strptime(s, '%m/%Y')
        except:
            pass
    parts = s.split()
    if len(parts) == 2:
        mes_abbr, ano = parts
        mes = mes_map.get(mes_abbr[:3])
        if mes:
            return datetime(int(ano), mes, 1)
    raise ValueError(f'Formato de data não reconhecido: {x!r}')

### 1. ETL da Base **Sell In**

In [13]:
sell_in = pd.read_excel(RAW_DIR / 'base sell in.xlsx')
sell_in.columns = sell_in.columns.str.strip().str.replace(r'[\r\n\t]+',' ', regex=True)

sell_in['Data_SellIn'] = sell_in['Calendario[Mês/Ano]'].apply(parse_mes_ano)
sell_in['Ano_Mes']      = sell_in['Data_SellIn'].dt.strftime('%Y-%m')

sell_in['[SumP_NF_ValTotal]']   = sell_in['[SumP_NF_ValTotal]'].replace(['-',''], np.nan).astype(float)
sell_in['[SumPeso_Liquido]']    = sell_in['[SumPeso_Liquido]'].replace(['-',''], np.nan).astype(float)
sell_in['Quantidade'] = (
    pd.to_numeric(sell_in['[SumP_NF_Quantidade]'], errors='coerce')
      .fillna(0)
      .astype(int)
)

sell_in.drop(columns='[SumP_NF_Quantidade]', inplace=True)

sell_in = sell_in[
    (
        (sell_in['FATO[P.NF.OperacaoFini]'] == 'Venda') 
    ) &
    (~sell_in['Produto[P.Produtos.LinhaProducao]'].str.contains('PROMOCIONAIS', case=False, na=False)) &
    (sell_in['[SumP_NF_ValTotal]'].notna())
].copy()

mask = (
    (sell_in['Produto[P.Produtos.LinhaProducao]']=='MARSHMALLOW') &
    sell_in['Produto[P.Produtos.TpPro.Inteligente]'].fillna('').eq('')
)
sell_in.loc[mask, 'Produto[P.Produtos.TpPro.Inteligente]'] = 'MARSH'

sell_in.rename(columns={
    'FATO[P.NF.Ramo_Atividade]':'Ramo_Atividade',
    'Cliente[P.Cliente.Pais]':'Pais','Cliente[P.Cliente.Regiao]':'Regiao',
    'Cliente[P.Cliente.NomeFantasia]':'Cliente','Cliente[P.Cliente.UF]':'UF',
    'Produto[P.Produtos.LinhaProducao]':'Linha_Producao','Produto[P.Produtos.Gramagem]':'Gramagem',
    'Produto[P.Produtos.MacroFormato]':'Macro_Formato','Produto[P.Produtos.Nome]':'Nome_Produto',
    'Produto[P.Produtos.Grupo]':'Grupo_Produto','Produto[P.Produtos.TpPro.Inteligente]':'Tipo_Produto',
    'Cliente[P.Cliente.Municipio]':'Municipio','Hierarquia[P.Cliente.Canal_Hierarquia]':'Canal',
    'Hierarquia[P.Cliente.GerenteRegional]':'Gerente_Regional','Hierarquia[P.Cliente.VendedorNome]':'Vendedor',
    'Produto[Cod. Produto]':'Cod_Produto','[SumP_NF_Quantidade]':'Quantidade',
    '[SumP_NF_ValTotal]':'Valor_Total','[SumPeso_Liquido]':'Peso_Liquido'
}, inplace=True)
print('Sell In pronto:', sell_in.shape)

Sell In pronto: (4172, 23)


C:\Users\frdsz\AppData\Local\Temp\ipykernel_23548\2507126671.py:7: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\frdsz\AppData\Local\Temp\ipykernel_23548\2507126671.py:8: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



### 2. ETL da Base **Sell Out**

In [14]:
sell_out = pd.read_excel(RAW_DIR / 'BASE SELL OUT.xlsx')
sell_out.columns = sell_out.columns.str.strip().str.replace(r'[\r\n\t]+',' ', regex=True)

sell_out['Data']    = pd.to_datetime(sell_out['Ano/Mês'], errors='raise')
sell_out['Ano_Mes'] = sell_out['Data'].dt.strftime('%Y-%m')

for col in ['Sell through valor','Sell through cx']:
    sell_out[col] = pd.to_numeric(
        sell_out[col].astype(str).str.replace(r'[^0-9,.-]','', regex=True).str.replace(',','.'), errors='coerce'
    )
sell_out['Sell through unidade'] = pd.to_numeric(sell_out['Sell through unidade'], errors='coerce').fillna(0).astype(int)

sell_out = sell_out[sell_out['Sell through valor']>0].copy()
sell_out.rename(columns={
    'Agente de Distribuição':'Agente_Distribuicao','UF PDV':'UF_PDV','Cidade PDV':'Cidade_PDV',
    'Segmentação Mtrix':'Segmentacao_Mtrix','Cód Produto (SKU)':'Cod_Produto','Nome Produto':'Nome_Produto',
    'Sell through valor':'Valor_SellThrough','Sell through unidade':'Unidades_SellThrough',
    'Sell through cx':'Caixas_SellThrough'
}, inplace=True)
print('Sell Out pronto:', sell_out.shape)

Sell Out pronto: (25708, 12)


### 3. Salvando Bases Tratadas

In [16]:
sell_in.to_csv(
    PROCESSED_DIR / "sell_in_processed.csv",
    sep=";",               # usa ponto-e-vírgula
    decimal=",",           # usa vírgula como decimal
    float_format="%.2f",   # sempre duas casas decimais
    index=False,
    encoding="utf-8-sig"   )

sell_out.to_csv(
    PROCESSED_DIR / "sell_out_processed.csv",
    sep=";",
    decimal=",",
    float_format="%.2f",
    index=False,
    encoding="utf-8-sig"
)
print('Bases tratadas salvas em:', PROCESSED_DIR)

Bases tratadas salvas em: c:\Users\frdsz\Downloads\Case Fini v0.1\data\processed


## 4. Análise Exploratória de Dados (EDA)
Nesta seção: KPIs globais, séries temporais, performance e métricas avançadas.

In [ ]:
# KPIs Globais
total_sell_in     = sell_in['Valor_Total'].sum()
total_box_sell_in = sell_in['Quantidade'].sum()
total_sell_out    = sell_out['Valor_SellThrough'].sum()
total_box_sell_out = sell_out['Caixas_SellThrough'].sum()
total_unidades_sell_out = sell_out['Unidades_SellThrough'].sum()
sell_through_rate = total_sell_out / total_sell_in
ticket_medio_out     = total_sell_out / sell_out['Unidades_SellThrough'].sum()
ticket_medio_in      = total_sell_in / sell_in['Quantidade'].sum()
crescimento_mensal = (
    sell_in.groupby('Ano_Mes')['Valor_Total'].sum()
      .pct_change().mul(100).round(2)
)

display(Markdown(f"**Total Sell In:**   R$ {total_sell_in:,.2f}"))
display(Markdown(f"**Total Caixas Sell In:** {total_box_sell_in:,.0f}"))
display(Markdown(f"**Total Sell Out:**  R$ {total_sell_out:,.2f}"))
display(Markdown(f"**Total Caixas Sell Out:** {total_box_sell_out:,.0f}"))
display(Markdown(f"**Total Unidades Sell Out:** {total_unidades_sell_out:,.0f}"))
display(Markdown(f"**Sell-Through Rate:** {sell_through_rate:.2%}"))
display(Markdown(f"**Ticket Médio:**     R$ {ticket_medio_out:,.2f}"))
display(Markdown(f"**Ticket Médio Sell In:** R$ {ticket_medio_in:,.2f}")) 
display(Markdown("**Crescimento Mensal de Sell In (%):**"))
display(crescimento_mensal.to_frame("Crescimento %"))


df_units = sell_out.groupby('Ano_Mes')['Unidades_SellThrough'].sum().reset_index()
df_monthly = df_sn.merge(df_so, on='Ano_Mes').merge(df_units, on='Ano_Mes')
df_monthly['SellThroughRate']   = df_monthly['SellOut'] / df_monthly['SellIn']
df_monthly['TicketMedio']       = df_monthly['SellOut'] / df_monthly['Unidades_SellThrough']
df_monthly['CrescimentoSellIn'] = df_monthly['SellIn'].pct_change().mul(100).round(2)
display(Markdown('**Visão Mensal das Métricas Avançadas**'))
display(df_monthly.style.format({
    'SellIn':'R$ {:,.0f}','SellOut':'R$ {:,.0f}',
    'SellThroughRate':'{:.2%}','TicketMedio':'R$ {:,.2f}',
    'CrescimentoSellIn':'{:.2f}%'
}))

**Total Sell In:**   R$ 106,518,219.36

**Total Caixas Sell In:** 2,514,105

**Total Sell Out:**  R$ 29,901,537.82

**Total Caixas Sell Out:** 475,671

**Total Unidades Sell Out:** 7,579,897

...

**Ticket Médio Sell In:** R$ 42.37

**Crescimento Mensal de Sell In (%):**

,Crescimento %
Ano_Mes,
2023-01,NaN
2023-02,57.61
2023-03,17.27
2023-04,9.57
2023-05,25.26
2023-06,-6.40
2023-07,-45.09
2023-08,79.93
2023-09,39.02


**Visão Mensal das Métricas Avançadas**

,Ano_Mes,SellIn,SellOut,Unidades_SellThrough,SellThroughRate,TicketMedio,CrescimentoSellIn
0,2023-01,"R$ 1,915,287","R$ 899,398",219804,46.96%,R$ 4.09,nan%
1,2023-02,"R$ 3,018,623","R$ 1,489,359",346704,49.34%,R$ 4.30,57.61%
2,2023-03,"R$ 3,539,933","R$ 1,143,771",247680,32.31%,R$ 4.62,17.27%
3,2023-04,"R$ 3,878,737","R$ 1,532,981",349572,39.52%,R$ 4.39,9.57%
4,2023-05,"R$ 4,858,471","R$ 1,214,656",282948,25.00%,R$ 4.29,25.26%
5,2023-06,"R$ 4,547,449","R$ 1,124,491",266730,24.73%,R$ 4.22,-6.40%
6,2023-07,"R$ 2,496,995","R$ 1,240,954",274364,49.70%,R$ 4.52,-45.09%
7,2023-08,"R$ 4,492,848","R$ 1,032,151",297633,22.97%,R$ 3.47,79.93%
8,2023-09,"R$ 6,245,769","R$ 1,404,807",355602,22.49%,R$ 3.95,39.02%
9,2023-10,"R$ 4,912,862","R$ 1,198,266",340700,24.39%,R$ 3.52,-21.34%


In [18]:
# Séries Temporais
df_sn = sell_in.groupby('Ano_Mes')['Valor_Total'].sum().reset_index().rename(columns={'Valor_Total':'SellIn'})
df_so = sell_out.groupby('Ano_Mes')['Valor_SellThrough'].sum().reset_index().rename(columns={'Valor_SellThrough':'SellOut'})
df_ts = df_sn.merge(df_so, on='Ano_Mes')
fig = px.line(df_ts, x='Ano_Mes', y=['SellIn','SellOut'], title='Evolução Mensal', labels={'value':'R$','Ano_Mes':'Mês/Ano','variable':'Métrica'})
fig.update_xaxes(tickangle=45)
fig.show()

In [19]:
# Top Produtos e Regiões
top_prod = sell_in.groupby('Tipo_Produto')['Valor_Total'].sum().nlargest(10).reset_index()
fig = px.bar(top_prod, x='Valor_Total', y='Tipo_Produto', orientation='h', title='Top 10 Produtos')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

by_region = sell_in.groupby('Regiao')['Valor_Total'].sum().reset_index()
fig2 = px.pie(by_region, names='Regiao', values='Valor_Total', title='Participação por Região')
fig2.show()

### Métricas Avançadas: Ticket Médio, Sell-Through Rate e Crescimento Mensal

In [20]:
df_units = sell_out.groupby('Ano_Mes')['Unidades_SellThrough'].sum().reset_index()
df_monthly = df_sn.merge(df_so, on='Ano_Mes').merge(df_units, on='Ano_Mes')
df_monthly['SellThroughRate']   = df_monthly['SellOut'] / df_monthly['SellIn']
df_monthly['TicketMedio']       = df_monthly['SellOut'] / df_monthly['Unidades_SellThrough']
df_monthly['CrescimentoSellIn'] = df_monthly['SellIn'].pct_change().mul(100).round(2)
display(Markdown('**Visão Mensal das Métricas Avançadas**'))
display(df_monthly.style.format({
    'SellIn':'R$ {:,.0f}','SellOut':'R$ {:,.0f}',
    'SellThroughRate':'{:.2%}','TicketMedio':'R$ {:,.2f}',
    'CrescimentoSellIn':'{:.2f}%'
}))
# Gráficos avançados
fig3 = px.line(df_monthly, x='Ano_Mes', y='SellThroughRate', title='Sell-Through Rate Mensal')
fig3.update_yaxes(tickformat='.0%'); fig3.update_xaxes(tickangle=45); fig3.show()
fig4 = px.line(df_monthly, x='Ano_Mes', y='TicketMedio', title='Ticket Médio Mensal'); fig4.update_xaxes(tickangle=45); fig4.show()
fig5 = px.bar(df_monthly, x='Ano_Mes', y='CrescimentoSellIn', title='Crescimento Mensal (%)'); fig5.update_xaxes(tickangle=45); fig5.show()

**Visão Mensal das Métricas Avançadas**

,Ano_Mes,SellIn,SellOut,Unidades_SellThrough,SellThroughRate,TicketMedio,CrescimentoSellIn
0,2023-01,"R$ 1,915,287","R$ 899,398",219804,46.96%,R$ 4.09,nan%
1,2023-02,"R$ 3,018,623","R$ 1,489,359",346704,49.34%,R$ 4.30,57.61%
2,2023-03,"R$ 3,539,933","R$ 1,143,771",247680,32.31%,R$ 4.62,17.27%
3,2023-04,"R$ 3,878,737","R$ 1,532,981",349572,39.52%,R$ 4.39,9.57%
4,2023-05,"R$ 4,858,471","R$ 1,214,656",282948,25.00%,R$ 4.29,25.26%
5,2023-06,"R$ 4,547,449","R$ 1,124,491",266730,24.73%,R$ 4.22,-6.40%
6,2023-07,"R$ 2,496,995","R$ 1,240,954",274364,49.70%,R$ 4.52,-45.09%
7,2023-08,"R$ 4,492,848","R$ 1,032,151",297633,22.97%,R$ 3.47,79.93%
8,2023-09,"R$ 6,245,769","R$ 1,404,807",355602,22.49%,R$ 3.95,39.02%
9,2023-10,"R$ 4,912,862","R$ 1,198,266",340700,24.39%,R$ 3.52,-21.34%
